## Задание 2

Состоит из **обязательной** и **бонусной** частей.

Обязательная часть оценивается в **50 баллов** и выполняется до **16 декабря 09:00**.

Бонусную часть можно делать, пока не придет необходимость получения оценки/зачета.

Обязательная часть заключается в fine-tuning несложной нейросети (ResNet-18) на UIUC Sports Event Dataset (http://vision.stanford.edu/lijiali/event_dataset) и ее последующем ускорении с помощью фреймворка **NVIDIA TensorRT** (https://developer.nvidia.com/tensorrt).

In [7]:
import torch
import torchvision
from torchvision import transforms, models

from tqdm import tqdm_notebook as tqdm

**(5 баллов)** Скачайте датасет, распакуйте его в директорию `./event_img/`. В ней должны оказаться 8 директорий, соответствующих классам картинок. Загрузите датасет в torch и разбейте случайным образом на train и val.

In [3]:
event_dataset = ...
assert isinstance(event_dataset, torch.utils.data.Dataset)

dataset_length = len(event_dataset)
print(f'Number of images: {dataset_length}')

Number of images: 1579


In [ ]:
val_size = .4
val_length = int(dataset_length * val_size)

train_data_raw, val_data_raw = ...

assert isinstance(train_data_raw, torch.utils.data.Dataset)
assert isinstance(val_data_raw, torch.utils.data.Dataset)

**(10 баллов)** Нам нужны разные преобразования (transforms) для train и val. Напишите класс `ApplyTransform`, объект которого — тот же датасет, что подается в конструкторе, но с примененными преобразованиями.

К `train_data_raw` нужно применить изменение размера до 256px (`min(height, width)`), извлечение региона 256x256 в центре, выбор в этом регионе случайного квадрата 224x224.

К `test_data_raw` нужно применить изменение размера до 224px и извлечение региона 224x224 в центре.

In [ ]:
class ApplyTransform(torch.utils.data.Dataset):
    
    def __init__(self, dataset, transform=None, target_transform=None):
        
        ...

In [ ]:
imagenet_normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

train_transform = transforms.Compose([
    ...
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    imagenet_normalize
])

val_transform = transforms.Compose([
    ...
    transforms.ToTensor(),
    imagenet_normalize
])

train_data = ApplyTransform(train_data_raw, train_transform)
val_data = ApplyTransform(val_data_raw, val_transform)

In [ ]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=8)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, num_workers=8)

**(5 баллов)** Загрузите предобученную на ImageNet модель ResNet-18, адаптируйте ее под классификацию на 8 классов. Создайте подходящую функцию потерь и оптимизатор SGD с `momentum=0.9`.

In [ ]:
num_classes = 8

model = ...
criterion = ...
optimizer = ...

**(10 баллов)** Реализуйте обучение модели, а потом измерьте время инференса на val датасете (`batch_size=32`).

In [ ]:
num_epochs = 10

for epoch in tqdm(range(num_epochs)):
    
    for X, y in train_loader:
        
        ...

In [ ]:
# measure inference time
...

**(20 баллов)** Установите TensorRT и **torch2trt** (https://github.com/NVIDIA-AI-IOT/torch2trt). Оптимизируйте с помощью torch2trt обученную модель и снова измерьте время инференса.

Попробуйте:
* как режим fp32, так и fp16
* не менее трех разных значений `batch_size`

In [8]:
from torch2trt import torch2trt

In [ ]:
...

### Бонусная часть (100 баллов)
Нужно переписать функцию `torch2trt()`, чтобы она поддерживала **режим `int8`-инференса**.

Сам TensorRT в режиме `int8` требует **калибровки**, которую нужно выполнять по train датасету. То есть нужно написать класс-калибратор (подкласс `tensorrt.IInt8EntropyCalibrator2`).

За уточнением того, что надо сделать, и за помощью обращаться к:
* https://docs.nvidia.com/deeplearning/sdk/tensorrt-developer-guide/index.html#enable_int8_python
* https://docs.nvidia.com/deeplearning/sdk/tensorrt-api/python_api/index.html
* примеру из TensorRT `samples/python/int8_caffe_mnist`
* преподавателю через почтовый ящик курса или telegram

In [ ]:
...